In [2]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2


In [14]:
import pandas
import cv2
import numpy
import tensorflow

In [7]:
data = pandas.read_csv("/Users/akshitshishodia/intern/labels_my-project-name_2023-06-27-09-45-54.csv")
data

,label_name,bbox_x,bbox_y,bbox_width,bbox_height,image_name,image_width,image_height
0,Gripper,574,498,102,130,frame0.jpg,1920,1080
1,Gripper,539,498,97,130,frame10.jpg,1920,1080
2,Gripper,350,590,74,120,frame100.jpg,1920,1080
3,Gripper,347,597,77,120,frame105.jpg,1920,1080
4,Gripper,345,585,100,130,frame110.jpg,1920,1080
...,...,...,...,...,...,...,...,...
139,Gripper,585,643,77,128,frame725.jpg,1920,1080
140,Gripper,388,615,92,123,frame80.jpg,1920,1080
141,Gripper,368,613,107,123,frame85.jpg,1920,1080
142,Gripper,370,600,77,133,frame90.jpg,1920,1080


In [10]:
from sklearn.model_selection import train_test_split
import os

In [11]:
class Tracker():

    H = 512
    W = 512
    def __init__(self) -> None:
        H = 512
        W = 512
    

    def build_model(input_shape):
        input = layers.Input(input_shape)

        pre_trained = MobileNetV2(
            include_top = False,
            weights = "imagenet",
            input_tensor = input,
            alpha = 1.0
        )
        pre_trained.trainable = False
        x = pre_trained.output
        x = layers.Conv2D(256, kernel_size=1, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(4, activation="sigmoid")(x)

        model = Model(input,x)
        return model

    def load_dataset(split = 0.2):
        data = pandas.read_csv('labels_my-project-name_2023-06-27-09-45-54.csv')
        path  = '/Users/akshitshishodia/intern/dataset1'
        image = list()
        bounding_box = list()

        for idx,rows in data.iterrows():
            name = path + str(rows['image_name'])
            bbox = [rows['bbox_x'],rows['bbox_y'],rows['bbox_width'],bbox['bbox_height']]
        
        sample_size = int(len(image)*split)

        train_x, valid_x = train_test_split(image,test_size=sample_size,random_state=42)
        train_y, valid_y = train_test_split(image,test_size=sample_size,random_state=42)

        train_x, test_x = train_test_split(train_x,test_size=sample_size,random_state=42)
        train_y, test_y = train_test_split(train_y, test_size=sample_size, random_state=42)

        return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)



    def read_image(self,path,bbox):
        
        img = cv2.imread(str(path),cv2.IMREAD_COLOR)
        height,width = img.shape
        img = cv2.reshape(img,(self.W,self.H))
        img = (img-127.5)/127.5
        img = img.astype(numpy.float32)

        x,y,w,h = bbox
        x_c = x+w
        y_c = y+h

        norm_x = float(x/width)
        norm_y = float(y/height)
        norm_xc = float(x_c/width)
        norm_yc = float(y_c/height)
        
        normalised = numpy.array([norm_x,norm_y,norm_xc,norm_yc])

        return normalised
    
    def parse(self,x,y):
        x,y = tensorflow.numpy_function(self.read_image,[x,y],[tensorflow.float32,tensorflow.float32])
        x.set_shape([self.H,self.W,3])
        y.set_shape([4])
        return x,y
    
    def tensor_dataset(self,image,b_box,batch=8):
        ds = tensorflow.data.Dataset.from_tensor_slices((image,b_box))
        ds = ds.map(self.parse).batch(batch).prefetch(10)
        return ds

    def run(self):
        
        





            

        




